In [1]:
cd /mnt/d/Dataset/Github/SCD/bbox

/mnt/d/Dataset/Github/SCD/bbox


In [2]:
ls

__init__.py*     csv/        notebooks/         utils.py*
__pycache__/     map.py*     requirements.txt*  zft2sub.py*
bbox_filter.py*  map_boxes/  sub2zft.py*


In [38]:
import pandas as pd, numpy as np
from tqdm import tqdm
df = pd.read_csv('csv/submission.csv')
df.head(2)

,id,PredictionString
0,000a312787f2_image,opacity 0.043109842436567326 773 811 1530 1955...
1,000c3a3f293f_image,opacity 0.04042375847365275 664 961 1327 1604 ...


In [5]:
# bboxes = np.array(df.PredictionString.iloc[0].strip().split(' ')).reshape(-1, 6)
# bboxes

array([['opacity', '0.043109842436567326', '773', '811', '1530', '1955'],
       ['opacity', '0.058176304744846294', '2361', '865', '3234', '1914'],
       ['opacity', '0.060274218741843494', '2860', '1104', '3450',
        '2269'],
       ['opacity', '0.09388556251809288', '590', '1342', '1263', '2473'],
       ['opacity', '0.0982183352252453', '2801', '1022', '3292', '1778'],
       ['opacity', '0.19193822209510328', '2535', '1008', '3350', '2207'],
       ['opacity', '0.1926010223888938', '632', '899', '1604', '2466'],
       ['none', '0.041312426328659', '0', '0', '1', '1']], dtype='<U20')

In [6]:
# data = []
# for idx in tqdm(range(df.shape[0])):
#     row = df.iloc[idx]
#     image_id = row["id"]
#     strings = np.array(row["PredictionString"].strip().split(' ')).reshape(-1, 6)
#     for string in strings:
#         data.append([image_id]+string.tolist())


    


100%|██████████| 12123/12123 [00:05<00:00, 2297.51it/s]


In [39]:
sub_df = pd.DataFrame(data, columns=["ImageID","LabelName","Conf","XMin","XMax","YMin","YMax"])

In [40]:
# !python3 sub2zft.py

progress : 100%|████████████████████████| 12123/12123 [00:06<00:00, 1971.27it/s]

              ImageID LabelName      Conf    XMin    XMax    YMin    YMax
0  000a312787f2_image   opacity  0.043110   773.0  1530.0   811.0  1955.0
1  000a312787f2_image   opacity  0.058176  2361.0  3234.0   865.0  1914.0
2  000a312787f2_image   opacity  0.060274  2860.0  3450.0  1104.0  2269.0
3  000a312787f2_image   opacity  0.093886   590.0  1263.0  1342.0  2473.0
4  000a312787f2_image   opacity  0.098218  2801.0  3292.0  1022.0  1778.0


In [11]:
sub_df = sub_df.astype(str)

In [45]:
from sub2zft import sub2zft
sub_df = sub2zft(df)
sub_df[["XMin","XMax","YMin","YMax"]] = sub_df[["XMin","XMax","YMin","YMax"]].astype(int)
sub_df = sub_df.astype(str)


progress : 100%|██████████| 12123/12123 [00:07<00:00, 1576.05it/s]


In [46]:
def get_predstring(row):
    string = ["1" if row['LabelName']=='opacity' else "0", row['Conf'], row['XMin'], row['YMin'], row['XMax'], row['YMax']]
    row['PredictionString'] = ' '.join(string)
    return row
sub_df = sub_df.iloc[:100].progress_apply(get_predstring, axis=1)
sub_df.head(2)

100%|██████████| 100/100 [00:00<00:00, 354.11it/s]


,ImageID,LabelName,Conf,XMin,XMax,YMin,YMax,PredictionString
0,000a312787f2_image,opacity,0.043109842436567326,773,1530,811,1955,1 0.043109842436567326 773 811 1530 1955
1,000a312787f2_image,opacity,0.058176304744846294,2361,3234,865,1914,1 0.058176304744846294 2361 865 3234 1914


In [49]:
pd.DataFrame(sub_df.groupby('ImageID')["PredictionString"].apply(list).progress_apply(lambda x: '\n'.join(x)).reset_index(drop=False)).rename({"ImageID":"id"}, axis=1)

100%|██████████| 11/11 [00:00<00:00, 8274.27it/s]


,id,PredictionString
0,000a312787f2_image,1 0.043109842436567326 773 811 1530 1955\n1 0....
1,000c3a3f293f_image,1 0.04042375847365275 664 961 1327 1604\n1 0.0...
2,0012ff7358bc_image,1 0.04592679627800743 2161 740 2465 1257\n1 0....
3,001398f4ff4f_image,1 0.03655236675280048 769 2035 1363 2661\n1 0....
4,001bd15d1891_image,1 0.05183601409349559 2629 853 3095 1559\n1 0....
5,0022227f5adf_image,1 0.041965300496344046 643 1369 1293 1974\n1 0...
6,0023f02ae886_image,1 0.036717522198602665 2257 874 2501 1220\n1 0...
7,002e9b2128d0_image,1 0.04511445523324106 224 1227 903 2053\n1 0.0...
8,00326161e51e_image,1 0.043801246284134086 543 401 1036 922\n1 0.0...
9,0044e449aae1_image,1 0.03748016998782539 587 287 1274 1405\n1 0.0...


In [17]:
help(sub2zft)

Help on function sub2zft in module sub2zft:

sub2zft(df)
    converts submission.csv to zfturbo format.
    Args:
        df : DataFrame (submission.csv)
    Returns:
        zfturbo formtted csv



In [15]:
import sys
del sys.modules['sub2zft']

In [37]:
sub_df

,ImageID,LabelName,Conf,XMin,XMax,YMin,YMax,PredictionString
0,000a312787f2_image,opacity,0.043109842436567326,773.0,1530.0,811.0,1955.0,1 0.043109842436567326 773.0 811.0 1530.0 1955.0
1,000a312787f2_image,opacity,0.058176304744846294,2361.0,3234.0,865.0,1914.0,1 0.058176304744846294 2361.0 865.0 3234.0 1914.0
2,000a312787f2_image,opacity,0.060274218741843494,2860.0,3450.0,1104.0,2269.0,1 0.060274218741843494 2860.0 1104.0 3450.0 22...
3,000a312787f2_image,opacity,0.09388556251809288,590.0,1263.0,1342.0,2473.0,1 0.09388556251809288 590.0 1342.0 1263.0 2473.0
4,000a312787f2_image,opacity,0.0982183352252453,2801.0,3292.0,1022.0,1778.0,1 0.0982183352252453 2801.0 1022.0 3292.0 1778.0
...,...,...,...,...,...,...,...,...
95,0049814626c8_image,opacity,0.041657879777014864,481.0,1120.0,1489.0,2508.0,1 0.041657879777014864 481.0 1489.0 1120.0 2508.0
96,0049814626c8_image,opacity,0.044654000216899685,714.0,1585.0,714.0,1665.0,1 0.044654000216899685 714.0 714.0 1585.0 1665.0
97,0049814626c8_image,opacity,0.046742170632937036,2796.0,3435.0,829.0,1910.0,1 0.046742170632937036 2796.0 829.0 3435.0 1910.0
98,0049814626c8_image,opacity,0.04859817822678338,539.0,1103.0,1135.0,2175.0,1 0.04859817822678338 539.0 1135.0 1103.0 2175.0
